In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from PIL import Image

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [4]:
#from tensorflow.keras.preprocessing import image_dataset_from_directory

In [5]:
from tensorflow.keras.optimizers import Adam, SGD

In [6]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
batch_size = 32

## vet all files can be workable with Tensorflow

In [9]:
# import pandas as pd
# import numpy as np
# df=pd.read_csv('../data/cleaned_data.csv')

In [10]:
# df

In [11]:
# train_datagen.flow_from_dataframe(
# )

In [12]:
dim = 150

In [13]:
train_generator = train_datagen.flow_from_directory(
        '../data/train_folder',  # this is the target directory
        target_size=(dim, dim),  # all images will be resized to 150x150
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 10285 images belonging to 12 classes.


In [14]:
validation_generator = test_datagen.flow_from_directory(
        '../data/validation_folder',
        target_size=(dim, dim),
        batch_size=batch_size,
        class_mode='categorical')

Found 1140 images belonging to 12 classes.


In [15]:
steps_per_epoch = 100
epoch  = 50
def my_gen(gen):
    i=0
    while i < steps_per_epoch * epoch:
        try:
            data, labels = next(gen)
            i+=1
            yield data, labels
        except:
            pass

In [16]:
from tensorflow.keras import backend as K
img_width, img_height = dim, dim
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [17]:
# from tensorflow.keras import callbacks
# my_callbacks = [
#     callbacks.EarlyStopping(monitor='loss', mode= 'min', patience=5),
#     callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0,
#     min_delta=0.001, cooldown=1, min_lr=0, mode = 'min')
# ]

In [18]:
from tensorflow.keras import callbacks

filepath="../checkpoints/little_model_weights_improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

my_callbacks = [
    callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.002, mode= 'min', patience=10),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=0,
    min_delta=0.01, cooldown=1, min_lr=0, mode = 'min'),
    callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, 
                              save_best_only=True, mode='max', save_weights_only = True)
    ]

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(input_shape)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), input_shape=(input_shape)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(12))
model.add(Activation('softmax'))

optimizer = SGD(lr=0.005)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 146, 146, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 128)       3

In [21]:
hist= model.fit(
        my_gen(train_generator),
        steps_per_epoch=steps_per_epoch,
        epochs=epoch,
        validation_data=validation_generator,callbacks=my_callbacks)

Epoch 1/50
 62/100 [=================>............] - ETA: 27s - loss: 2.4863 - accuracy: 0.0832

KeyboardInterrupt: 

In [ ]:
model.save('../models/little_model.h5')

In [ ]:
# test_loss, test_acc = model.evaluate(X_test_f, y_test_f, verbose=0)

# print('\nTest accuracy {:5.2f}%'.format(100*test_acc))

In [ ]:
training_loss = hist.history['loss']
test_loss = hist.history['val_loss']

epoch_count = range(1,len(training_loss)+1)
fig,ax = plt.subplots(figsize=(15,8))

sns.set(font_scale=1.15)
sns.lineplot(x=epoch_count,y=training_loss,ax=ax)
sns.lineplot(x=epoch_count,y=test_loss,ax=ax)

ax.set_title('Loss Curves',fontsize=19)
ax.set_ylabel('Loss',fontsize=18)
ax.set_xlabel('Epochs',fontsize=18)

plt.legend(['Training Loss', 'Validation Loss'])
#plt.xlabel('Epoch')
#plt.ylabel('Loss')
#plt.title('Training and Validation Loss Curves')
plt.show();
plt.savefig('../data/images/little_model.png')

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

epoch_count = range(1,len(training_loss)+1)

fig,ax = plt.subplots(figsize=(15,8))
sns.set(font_scale=1.15)
sns.lineplot(x=epoch_count,y=acc,ax=ax)
sns.lineplot(x=epoch_count,y=val_acc,ax=ax)

ax.set_title('Accuracy Curves',fontsize=19)
ax.set_ylabel('Accuracy',fontsize=18)
ax.set_xlabel('Epochs',fontsize=18)

plt.legend(['Training Accuracy', 'Validation Accuracy'])

# plt.ylabel('Accuracy')
# plt.ylim([min(plt.ylim()),1])
# plt.title('Training and Validation Accuracies')
#plt.xlabel('Epoch')
plt.show();
plt.savefig('../data/images/little_model.png')

In [ ]:
model.evaluate()